# Load vào dataset có sẵn trên kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_35.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_42.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_16.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_17.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_22.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_54.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_18.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_62.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_68.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_44.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_53.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_38.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_3.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_28.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_27.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_55.jpg
/kaggle/input/thyroidcancer-ver1/ver1/valid/B6/B6_33.jpg
/kaggle/input/thyroidcancer-ver1

# Tải xuống tài nguyên code dự án và update

In [2]:
%cd /kaggle/working

/kaggle/working


In [3]:
!git clone https://github.com/Harito97/ThyroidCancerClassifier

Cloning into 'ThyroidCancerClassifier'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 442 (delta 44), reused 68 (delta 30), pack-reused 355
Receiving objects: 100% (442/442), 1.35 MiB | 9.70 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [4]:
%cd ThyroidCancerClassifier
%ls

/kaggle/working/ThyroidCancerClassifier
README.md  guide.ipynb       move_mouse.py  requirements.txt
data/      models/           notebooks/     src/
docs/      move_mouse.ipynb  ref_paper/     temp.ipynb


In [5]:
!git pull

Already up to date.


# Khai báo data_dir sẽ làm việc và wandb để lưu lịch sử

In [6]:
%pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [7]:
data_dir = '/kaggle/input/thyroidcancer-ver1/ver1'

In [8]:
# Log in to your W&B account
import wandb

In [9]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Với Model h97_resnet_B2_B5B6_dataver1

In [10]:
from src.model.classifier.H97 import H97_ResNet
import torch
import os
from src.experiment.load_data import load_data

model = H97_ResNet(num_classes=2)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 142MB/s] 


In [11]:
import torch
from torchsummary import summary

# Assuming `model` is an instance of H97_ResNet or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct usage of summary from torchsummary
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [12]:
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['.B2'], 1: ['B5', 'B6']})

Creating dataset...
Balancing the dataset
Original dataset size: 1150 | 1150
Balanced dataset size: 4272 | 4272
Train dataset size: 4272
Valid dataset size: 130
Test dataset size: 145
Creating dataloader...
Train loader size: 134
Valid loader size: 5
Test loader size: 5


In [13]:
for name, loader in zip(['train', 'valid', 'test'], [train_loader, valid_loader, test_loader]):
    print(f'Counting labels for {name}...')
    labels = [0, 0]
    for _, target in loader:
        labels[0] += torch.sum(target == 0).item()
        labels[1] += torch.sum(target == 1).item()
    print(f'Class 0: {labels[0]} images')
    print(f'Class 1: {labels[1]} images')

Counting labels for train...
Class 0: 2136 images
Class 1: 2136 images
Counting labels for valid...
Class 0: 10 images
Class 1: 120 images
Counting labels for test...
Class 0: 11 images
Class 1: 134 images


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
!mkdir -p /kaggle/working/h97_resnet_B2_B5B6_dataver1

In [ ]:
from src.experiment.train_model import train_model

train_model(train_loader, valid_loader, model, device, criterion, optimizer, 100, 30, '/kaggle/working/h97_resnet_B2_B5B6_dataver1', 'h97_resnet_B2_B5B6_dataver1')

wandb: Currently logged in as: harito97 (harito). Use `wandb login --relogin` to force relogin


Moving model, criterion, optimizer to device ...
Taking model_destination, model_name and history_file_path ...
Initializing history ...
Training classification model...
Epoch 1/100
Progress: 100.00%
Calculating metrics in training set ...

Start validation at epoch 1 ...
Train Loss: 0.504214, Train Acc: 0.761236, Train F1: 0.758652
Val   Loss: 0.282190, Val   Acc: 0.969231, Val   F1: 0.970500
Saved last history at epoch 1
Saved **best model** at epoch 1
Epoch 2/100
Progress: 100.00%
Calculating metrics in training set ...

Start validation at epoch 2 ...
Train Loss: 0.328983, Train Acc: 0.858146, Train F1: 0.857569
Val   Loss: 0.130539, Val   Acc: 0.976923, Val   F1: 0.977424
Saved last history at epoch 2
Saved **best model** at epoch 2
Epoch 3/100
Progress: 100.00%
Calculating metrics in training set ...

Start validation at epoch 3 ...
Train Loss: 0.282819, Train Acc: 0.872893, Train F1: 0.872446
Val   Loss: 0.107610, Val   Acc: 0.976923, Val   F1: 0.977424
Saved last history at epo

In [ ]:
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_resnet_B2_B5B6_dataver1",
    model_name="h97_resnet_B2_B5B6_dataver1",
)

Sau khi train xong nhanh chóng tải xuống kết quả

# Với Model h97_resnet_B5_B2B6_dataver1

In [ ]:
from src.model.classifier.H97 import H97_ResNet
import torch
import os
from src.experiment.load_data import load_data

model = H97_ResNet(num_classes=2)

In [ ]:
import torch
from torchsummary import summary

# Assuming `model` is an instance of H97_ResNet or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct usage of summary from torchsummary
summary(model, input_size=(3, 224, 224))

In [ ]:
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['B5'], 1: ['.B2', 'B6']})

In [ ]:
for name, loader in zip(['train', 'valid', 'test'], [train_loader, valid_loader, test_loader]):
    print(f'Counting labels for {name}...')
    labels = [0, 0]
    for _, target in loader:
        labels[0] += torch.sum(target == 0).item()
        labels[1] += torch.sum(target == 1).item()
    print(f'Class 0: {labels[0]} images')
    print(f'Class 1: {labels[1]} images')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
!mkdir -p /kaggle/working/h97_resnet_B5_B2B6_dataver1

In [ ]:
from src.experiment.train_model import train_model

train_model(train_loader, valid_loader, model, device, criterion, optimizer, 100, 30, '/kaggle/working/h97_resnet_B5_B2B6_dataver1', 'h97_resnet_B5_B2B6_dataver1')

In [ ]:
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_resnet_B5_B2B6_dataver1",
    model_name="h97_resnet_B5_B2B6_dataver1",
)

Sau khi train xong nhanh chóng tải xuống kết quả

# Với Model h97_resnet_B6_B2B5_dataver1

In [ ]:
from src.model.classifier.H97 import H97_ResNet
import torch
import os
from src.experiment.load_data import load_data

model = H97_ResNet(num_classes=2)

In [ ]:
import torch
from torchsummary import summary

# Assuming `model` is an instance of H97_ResNet or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct usage of summary from torchsummary
summary(model, input_size=(3, 224, 224))

In [ ]:
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['B6'], 1: ['.B2', 'B5']})

In [ ]:
for name, loader in zip(['train', 'valid', 'test'], [train_loader, valid_loader, test_loader]):
    print(f'Counting labels for {name}...')
    labels = [0, 0]
    for _, target in loader:
        labels[0] += torch.sum(target == 0).item()
        labels[1] += torch.sum(target == 1).item()
    print(f'Class 0: {labels[0]} images')
    print(f'Class 1: {labels[1]} images')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
!mkdir -p /kaggle/working/h97_resnet_B6_B2B5_dataver1

In [ ]:
from src.experiment.train_model import train_model

train_model(train_loader, valid_loader, model, device, criterion, optimizer, 100, 30, '/kaggle/working/h97_resnet_B6_B2B5_dataver1', 'h97_resnet_B6_B2B5_dataver1')

In [ ]:
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_resnet_B6_B2B5_dataver1",
    model_name="h97_resnet_B6_B2B5_dataver1",
)

Sau khi train xong nhanh chóng tải xuống kết quả

# Với Model h97_resnet_B2_B6_B5_dataver1

In [ ]:
from src.model.classifier.H97 import H97_ResNet
import torch
import os
from src.experiment.load_data import load_data

model = H97_ResNet(num_classes=3)

In [ ]:
import torch
from torchsummary import summary

# Assuming `model` is an instance of H97_ResNet or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct usage of summary from torchsummary
summary(model, input_size=(3, 224, 224))

In [ ]:
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['.B2'], 1: ['B5'], 2: ['B6']})

In [ ]:
for name, loader in zip(['train', 'valid', 'test'], [train_loader, valid_loader, test_loader]):
    print(f'Counting labels for {name}...')
    labels = [0, 0]
    for _, target in loader:
        labels[0] += torch.sum(target == 0).item()
        labels[1] += torch.sum(target == 1).item()
    print(f'Class 0: {labels[0]} images')
    print(f'Class 1: {labels[1]} images')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
!mkdir -p /kaggle/working/h97_resnet_B2_B6_B5_dataver1

In [ ]:
from src.experiment.train_model import train_model

train_model(train_loader, valid_loader, model, device, criterion, optimizer, 100, 30, '/kaggle/working/h97_resnet_B2_B6_B5_dataver1', 'h97_resnet_B2_B6_B5_dataver1')

In [ ]:
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_resnet_B2_B6_B5_dataver1",
    model_name="h97_resnet_B2_B6_B5_dataver1",
)

Sau khi train xong nhanh chóng tải xuống kết quả